<u><h1 style="color:#21B6A8"> Exercice : Dataset cleaning and mapinulation </h1></u>

<h2 style="color:#98D7C2">Importing library 🐼</h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

<h2 style="color:#98D7C2">Reading datasets</h2>

In [2]:
customers = pd.read_csv('dataset/customers.csv')
products = pd.read_csv('dataset/products.csv')
transactions = pd.read_csv('dataset/transactions.csv')

<h2 style="color:#98D7C2">Copy creations</h2>

In [3]:
customers_copie = customers.copy()
products_copie = products.copy()
transactions_copie = transactions.copy()

<i style="color:#90ADC6"> -> je crée des copy des fichiers csv originaux car je ne veux pas écraser l'original.  </i>

<h2 style="color:#98D7C2">Displaying data </h2>

In [5]:
customers_copie.head()
products_copie.head() 
transactions_copie.head()

id_prod                        date session_id client_id
0  0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450
1   2_226  2022-02-03 01:55:53.276402   s_159142     c_277
2   1_374  2021-09-23 15:13:46.938559    s_94290    c_4270
3  0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597
4  0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242

<h2 style="color:#98D7C2">Data description</h2>

In [ ]:
transactions_copie.describe()

<i style="color:#90ADC6"> -> ici, je vois que la valeur "test" revient trop souvent dans la colonne date, ce n’est pas une date valide, pandas considère toute la colonne comme un objet. Après avoir analysé les lignes avec cette valeur, je décide de "supprimer" ou plutôt de filtrer ces lignes car ca ressemble à des valeurs de test. </i>

<h2 style="color:#98D7C2">Deleting ‘test’ values</h2>

In [ ]:
transactions_copie = transactions_copie[~transactions_copie['date'].astype(str).str.contains("test", case=False, na=False)]

<h2 style="color:#98D7C2">Checking data types</h2>

In [ ]:
transactions_copie.dtypes

<i style="color:#90ADC6"> -> La date est au format objet, ce n'est pas correct, je vais donc la convertir en datetime. </i>

<h2 style="color:#98D7C2">Converting date to datetime</h2>

In [ ]:
transactions_copie.loc[:, 'date'] = pd.to_datetime(transactions_copie['date'])

<h2 style="color:#98D7C2">Checking for duplicates</h2>

In [ ]:
transactions_copie.duplicated().sum()
products_copie.duplicated().sum()
customers_copie.duplicated().sum()

<h2 style="color:#98D7C2">Checking for null values</h2>

In [ ]:
transactions_copie.isnull().sum()
products_copie.isnull().sum()
customers_copie.isnull().sum()

<h2 style="color:#98D7C2">Correction negative values</h2>

In [14]:
products_copie.describe()

price        categ
count  3287.000000  3287.000000
mean     21.856946     0.370246
std      29.847680     0.615387
min       0.000000     0.000000
25%       6.990000     0.000000
50%      13.060000     0.000000
75%      22.990000     1.000000
max     300.000000     2.000000

In [ ]:
products_copie['price'].min()

In [13]:
products_copie['price'] = products_copie['price'].replace(-1,0) 

<i style="color:#90ADC6"> -> Je m'aperçois que il y a une valeur négative dans la colonne price, c'est incorrect car techniquement ça n'existe pas un produit à -1 euro, donc je remplace cette valeur par 0. </i>

<h2 style="color:#98D7C2">Display monthly revenue trend</h2>

In [ ]:
merge_transac_products = pd.merge(transactions_copie, products_copie[['price', 'id_prod']], on='id_prod')
merge_transac_products['month'] = merge_transac_products['date'].dt.to_period('M')
ca_mois = merge_transac_products.groupby('month')['price'].sum().reset_index()
ca_mois

<i style="color:#90ADC6"> -> Je choisis de faire une jointure interne car je souhaite uniquement conserver les transactions pour lesquelles le produit existe dans la table des produits. </i>

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(ca_mois['month'].astype(str), ca_mois['price'], marker='x', linestyle='-')
plt.title("Chiffre d'affaires par mois")
plt.xlabel("Mois")
plt.ylabel("Chiffre d'affaires (€)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

<h2 style="color:#98D7C2">Calculates the number of transactions per customer</h2>

In [ ]:
merge_transac_customer = pd.merge(transactions_copie, customers_copie, on='client_id')
transac_by_customer = merge_transac_customer.groupby('client_id')['client_id'].count().reset_index(name='nmb_transactions')

<i style="color:#90ADC6"> -> Je fais une jointure interne car je veux afficher seulement les clients qui ont des transactions. Sans la jointure, je peux afficher 1 seule transaction par client or que moi je veux toutes les transactions qu'un client a fait donc la jointure est nécessaire ici.  </i>

<h2 style="color:#98D7C2">Calculation of revenue per customer</h2>

In [ ]:
merge_transac_customer_products = pd.merge(merge_transac_products, customers, on='client_id')
ca_by_client = merge_transac_customer_products.groupby('client_id')['price'].sum().reset_index(name='revenue')
ca_by_client

<h2 style="color:#98D7C2">Display of the person with the highest turnover</h2>

In [ ]:
ca_by_client.sort_values(by='revenue', ascending=False).head(1)

<h2 style="color:#98D7C2">Displaying the most expensive product</h2>

In [ ]:
products_copie.sort_values(by='price', ascending=False).head(1)

In [ ]:
products_copie.nlargest(1, 'price')

<h2 style="color:#98D7C2">Display of the best-selling product</h2>

In [ ]:
best_selling_product = merge_transac_products['id_prod'].value_counts().head(1)
best_selling_product

In [ ]:
je_test_les_nulle_une_seule_fois = je_test_les_nulle[je_test_les_nulle.isna().sum(axis=1) == 1]